In [1]:
import os

In [2]:
%pwd

'c:\\Users\\aarav\\Downloads\\IRIS-MLops\\research'

In [3]:
os.chdir('../')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('artifacts/data_ingestion/IRIS.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [7]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str 
    unzip_data_dir: Path 
    all_schema: dict 

In [34]:
# Configuration Manager

In [8]:
from IRIS_MLops.constants import *
from IRIS_MLops.utils.common import read_yaml,create_directories 

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
          config = self.config.data_validation 
          schema = self.schema.COLUMNS

          create_directories([config.root_dir])

          data_validation_config = DataValidationConfig(
                root_dir=config.root_dir,
                STATUS_FILE=config.STATUS_FILE,
                unzip_data_dir= config.unzip_data_dir,
                all_schema=schema,
          )
          
          return data_validation_config

In [29]:
# Components

In [10]:
import os
from IRIS_MLops import logger  

In [11]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 

    
    def validate_all_columns(self)-> bool:
        try:
            validation_status: None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True 
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e 

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e  

[2024-01-13 00:41:56,363: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-13 00:41:56,366: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-13 00:41:56,374: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-13 00:41:56,378: INFO: common: created directory at: artifacts]
[2024-01-13 00:41:56,381: INFO: common: created directory at: artifacts/data_validation]
